In [ ]:
import cv2
import numpy as np
import os

def calibrate(ldir, rdir, savedir, sq_size, bd_size):
    
    sq_points = np.zeros((1, bd_size[0] * bd_size[1], 3), np.float32)
    sq_points[0,:,:2] = (np.mgrid[0:bd_size[0], 0:bd_size[1]]*sq_size).T.reshape(-1, 2)
    
    obj_points = []
    img_points_l = []
    img_points_r = []
    
    l_files = sorted(os.listdir(ldir))
    r_files = sorted(os.listdir(rdir))
    
    for path_l, path_r in zip(l_files, r_files):
        
        img_l_path = os.path.join(ldir, path_l)
        img_r_path = os.path.join(rdir, path_r)
        
        img_l = cv2.imread(img_l_path)
        img_r = cv2.imread(img_r_path)
        
        if img_l is None or img_r is None:
            print(f"Failed to load images: {img_l_path} or {img_r_path}")
            continue
        
        h, w = img_l.shape[:2]
        gray_l = cv2.cvtColor(img_l, cv2.COLOR_BGR2GRAY)
        gray_r = cv2.cvtColor(img_r, cv2.COLOR_BGR2GRAY)
        
        ret_l, corners_l = cv2.findChessboardCornersSB(gray_l, bd_size, 
            cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE +\
            cv2.CALIB_CB_EXHAUSTIVE + cv2.CALIB_CB_ACCURACY)
        ret_r, corners_r = cv2.findChessboardCornersSB(gray_r, bd_size, 
            cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE +\
            cv2.CALIB_CB_EXHAUSTIVE + cv2.CALIB_CB_ACCURACY)
        
        if not ret_l:
            print(f"No checkerboard corners found in left image: {img_l_path}")
            continue
        if not ret_r:
            print(f"No checkerboard corners found in right image: {img_r_path}")
            continue

        winSize = (5, 5)
        zeroZone = (-1, -1)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TermCriteria_COUNT, 40, 0.001)
        
        corners_l = cv2.cornerSubPix(gray_l, corners_l, winSize, zeroZone, criteria)
        corners_r = cv2.cornerSubPix(gray_r, corners_r, winSize, zeroZone, criteria)
        
        obj_points.append(sq_points)
        img_points_l.append(corners_l)
        img_points_r.append(corners_r)

    ret_l, mtx_l, dist_l, rvecs_l, tvecs_l = cv2.calibrateCamera(obj_points, img_points_l, (w, h), None, None)
    ret_r, mtx_r, dist_r, rvecs_r, tvecs_r = cv2.calibrateCamera(obj_points, img_points_r, (w, h), None, None)
    
    ret_s, mtx_l, dist_l, mtx_r, dist_r, R, T, E, F = cv2.stereoCalibrate(obj_points, img_points_l, 
        img_points_r, mtx_l, dist_l, mtx_r, dist_r, (w, h), None, cv2.CALIB_FIX_INTRINSIC)
    
    print("Stereo calibration error: ", ret_s)
    
    Rr_l, Rr_r, P_l, P_r, Q, roi_l, roi_r = cv2.stereoRectify(mtx_l, dist_l, mtx_r, dist_r, 
        (w, h), R, T, 1, (0, 0))
    
    
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    
    
    np.save(os.path.join(savedir, "mtx_l.npy"), mtx_l)
    np.save(os.path.join(savedir, "dist_l.npy"), dist_l)
    np.save(os.path.join(savedir, "mtx_r.npy"), mtx_r)
    np.save(os.path.join(savedir, "dist_r.npy"), dist_r)
    np.save(os.path.join(savedir, "Rr_l.npy"), Rr_l)
    np.save(os.path.join(savedir, "P_l.npy"), P_l)
    np.save(os.path.join(savedir, "Rr_r.npy"), Rr_r)
    np.save(os.path.join(savedir, "P_r.npy"), P_r)
    np.save(os.path.join(savedir, "Q.npy"), Q)

    return mtx_l, dist_l, mtx_r, dist_r, Rr_l, P_l, Rr_r, P_r, Q




In [ ]:
import cv2
import numpy as np


left_images_dir = 'left_images/'
right_images_dir = 'right_images/'
save_dir = 'calibration_results/' 

square_size = 0.025  # 棋盘格每个方块的边长（单位：米）
board_size = (9, 6)  # 棋盘格的尺寸，表示棋盘上内部交点的数量（宽度，长度）


mtx_l, dist_l, mtx_r, dist_r, Rr_l, P_l, Rr_r, P_r, Q = calibrate(
    left_images_dir, right_images_dir, save_dir, square_size, board_size
    )

# 加载校准参数并矫正图像
# mtx_l = np.load('calibration_results/mtx_l.npy')
# dist_l = np.load('calibration_results/dist_l.npy')
# mtx_r = np.load('calibration_results/mtx_r.npy')
# dist_r = np.load('calibration_results/dist_r.npy')